In [17]:
from transformers import BartTokenizer, BartModel
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.layers import Dense,Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
#from tensorflow.keras import regularizer
from sklearn.model_selection import train_test_split
import os

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartModel.from_pretrained('facebook/bart-large')

max_len = 30 

def bert_preproc (sentences):
    # Prepare the model input
    input_ids = []
    attention_masks = []

    for sent in sentences:
        dbert_inps = tokenizer.encode_plus(sent, add_special_tokens=True, max_length=max_len,
                                                 pad_to_max_length=True, return_attention_mask=True, truncation=True)
        input_ids.append(dbert_inps['input_ids'])
        attention_masks.append(dbert_inps['attention_mask'])

    input_ids = np.asarray(input_ids)
    attention_masks = np.array(attention_masks)
    return input_ids, attention_masks


class BERT_Classification:
    def __init__(self):

        self.loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        self.metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
        self.model = model


    def bert_train (self, workspace, train_inp, train_mask, train_label, val_inp, val_mask, val_label,num_epochs):
        log_dir = workspace + os.sep + 'log'
    
        if os.path.exists(log_dir) and os.path.isdir(log_dir):
            shutil.rmtree(log_dir)

        os.makedirs(log_dir, exist_ok=True)
        model_save_path = workspace + os.sep +  'dbert_model.h5'
        chkpt = ModelCheckpoint(filepath = model_save_path, save_weights_only=True, monitor='val_loss',
                                               mode='min', save_best_only=True)
        tboard = TensorBoard(log_dir=log_dir)
        callbacks = [chkpt, tboard]

        self.model.compile(loss=self.loss, optimizer=self.optimizer, metrics=[self.metric])
        history = self.model.fit([train_inp, train_mask], train_label, batch_size=16, epochs=num_epochs,
                            validation_data=([val_inp, val_mask], val_label), callbacks=callbacks)
        return history


    def bert_predict (self, workspace, val_inp, val_mask):
        model_save_path = workspace + os.sep + 'dbert_model.h5'
        trained_model = bert_model()
        trained_model.compile(loss=self.loss, optimizer=self.optimizer, metrics=[self.metric])
        trained_model.load_weights(model_save_path)
        preds = trained_model.predict([val_inp, val_mask], batch_size=16)
        pred_labels = preds.argmax(axis=1)
        return pred_labels, preds.max(axis=1)


if __name__ == "__main__":
    workspace="tmp"
    file_path = r"C:\Users\jayanti.prasad\Data\NLP_DATA\train\IMDB_reviews.csv" 
    df = pd.read_csv(file_path, encoding='utf-8',nrows=1000)
    print(df.columns, df.shape)

    texts = df['text'].to_list()
    labels = df['label'].to_list()
    print(labels)
    
    input_ids, attention_masks = bert_preproc (texts)

    M =  BERT_Classification()

    print(M.model)

    train_inp, val_inp, train_label, val_label, train_mask, val_mask  = train_test_split(input_ids, labels,attention_masks, test_size=0.2)

    history = M.bert_train(workspace, train_inp, train_mask, train_label, val_inp, val_mask, val_label,10)
    
    plot_history(history)


Index(['Unnamed: 0', 'text', 'label'], dtype='object') (1000, 3)
[0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0

C:\Users\jayanti.prasad\Anaconda3\envs\aiml_env\lib\site-packages\transformers\tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


BartModel(
  (shared): Embedding(50265, 1024, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): Embedding(50265, 1024, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-11): 12 x BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layerno

AttributeError: 'BartModel' object has no attribute 'compile'